# Création de la base de données de following en utilisant l'api V2 de Twitter

In [ ]:
!pip install twitter==2.0a2

### Import des packages

In [ ]:
import twitter
import json
import time
from twitter import OAuth2
from urllib.request import urlopen

### On récupère la liste des ids à partir de laquelle on va construire le graphe

In [ ]:
# l'url change régulièrement, il faut donc aller à l'adresse suivante :
# https://github.com/jveillon/Projet-Python/raw/FixedScrapping/users.json 
# et rentrer l'url obtenu (contenant un token)
url = pass

response = urlopen(url)

data_json = json.loads(response.read())

ids = data_json['id']
liste_id = [ids[str(i)] for i in range(len(ids))]

### Les requêtes obtenues vont être stockées dans un dictionnaire : dictionnaire_following

In [ ]:
dictionnaire_following = {}
n_requete = len(liste_id)

### Fonction permettant de construire le dictionnaire

In [ ]:
def construction_dictionnaire(bearer,i) :
    '''
    Le but de cette fonction est de construire dictionnaire_following en utilisant plusieurs bearer différents, car
    chaque bearer est limité en termes de nombre de requêtes toutes les 15 minutes
    
    Entrée :
    
    bearer : str
    i : int : indice dans la boucle, pour pouvoir construire le dictionnaire sur de nouveaux utilisateurs
    --------------------
    Sortie :
    
    None
    --------------------
    Modifie par effet de bord dictionnaire_following
    
    '''
    # avec un compte, on ne peut faire que 15 demandes de listes de followers toutes les 15 minutes
    for j in range (i,i+15) :
        
        auth = OAuth2(bearer_token=bearer)
        api = twitter.Twitter(auth=auth,api_version="2",format="")
        
        id_user = liste_id[j]
        
        # la requête permettant d'accéder aux followings
        following_dict = api.users._id.following(_id= id_user,max_results=1000)
        
        # on doit vérifier que la requête obtenue est bien conforme
        # il peut arriver que le dictionnaire renvoyé en requête n'est pas bon 
        # (c'est le cas de l'id n°92 par exemple)
        if 'data' in following_dict.keys() :
            following = following_dict['data']
            # la liste des following de l'id en question
            ids_following = [following[k]['id'] for k in range(len(following))]
            dictionnaire_following.update({id_user : ids_following})

### Les différents bearers utilisés

In [ ]:
bearer_florent = 'AAAAAAAAAAAAAAAAAAAAACYpkQEAAAAAXEAZh5Grz9%2FdDc%2FAnqZmf%2BEsQqA%3Dz3k2j6XQA3M9XVp4yswCtj8gcd1Yi29sv1AER7YE9hEtqUZgSu'
bearer_ambroise = 'AAAAAAAAAAAAAAAAAAAAANXkXgEAAAAA9btVv9h0f4Gx8r3D8bE7Mto0C84%3DvqFXOjroHrNOVZwxQUyOBtuo9M7hCgcmR3IMOZTBhi1oIaHyeV'
bearer_florent2 = 'AAAAAAAAAAAAAAAAAAAAACrYkQEAAAAAjM1pHm9dab%2FXSJSm3V7NIkj8Rvk%3DTXHTMG9hgEuBax6g4w7zWH0dzCfR3VcqggeJMJU4yr03LG3uZ0'
bearer_florent3 = 'AAAAAAAAAAAAAAAAAAAAAIHYkQEAAAAAXWTfErOqO1gdiogRNk28XuHKsXE%3DoooFseB0tDcIM9v4DNQ6HDuP5zZZl4rZ9Xxrm9X5wnKtqFtGSM'
bearer_florent4 = 'AAAAAAAAAAAAAAAAAAAAAPbYkQEAAAAAoNVzBIY7RpXfu3aXsg3oqrHOKzw%3DbHKCEFRS3SUYRpahfwAM4jcD8mKRixUgDU2ohvheZuvrDD81BM'
bearer_florent5 = 'AAAAAAAAAAAAAAAAAAAAADrZkQEAAAAAz7qAPtcp5EqGZrwSja2070mQd6Y%3DlBQKfbpqjfdmkPDVDVybeWZUHk3X2VpCRMWFsQMtadPGTz81gU'
bearer_florent6 = 'AAAAAAAAAAAAAAAAAAAAAJrZkQEAAAAAzo8uJ4z03w8RIKj87APUPsEWfsk%3Dc9PxyfBThkJWisa9tebcoAlQ2d7DntnqzDP4ATmy6MGuEspIba'
bearer_xavier = 'AAAAAAAAAAAAAAAAAAAAAIvZkQEAAAAAJ8DFFWNxJwqfyjlRo3ibEzWXLLs%3DVf55ZgTYRp0HJmyekQRtaGBwm7ymyCONpcNFDIJa8o4Rwhs9v9'
bearer_juliette1 = 'AAAAAAAAAAAAAAAAAAAAAFiEjQEAAAAA6626vtqrbtrHkoIZ8x3XFJQWz6g%3DBYaIyb7sKLrOnlK02O3sbzUSYbF4rfigGpPScK4dflcGVDyDw4'
bearer_juliette2 = 'AAAAAAAAAAAAAAAAAAAAAD%2FdkQEAAAAABpVGLzCZXxT6Jm%2FY5Xw8DCQlamI%3Dyf8EylbyIbbLiZrsnOYfXVANirxBrl2K63JkjecH9bmtLNJPsh'
bearer_juliette3 = 'AAAAAAAAAAAAAAAAAAAAAHTdkQEAAAAAejnB8%2Bf89VPal%2FMQuaGEyKBbvlk%3DjNFRQtWJVPEHv9bUNVNNJ3IpJAuE9gN81dKvVKnCPSmqRxCLWL'

In [ ]:
bearers = [bearer_florent,
           bearer_ambroise,
           bearer_florent2,
           bearer_florent3,
           bearer_florent4,
           bearer_florent5,
           bearer_florent6,
           bearer_xavier,
           bearer_juliette1,
           bearer_juliette2,
           bearer_juliette3]

### Création du dictionnaire

In [ ]:
i = 0
continuer = True

while True :
        
    for bearer in bearers :
        print(i)
        if i >= n_requete - 15 :
            continuer = False
            break
        construction_dictionnaire(bearer,i)
        i += 15
    
    # on sauvegarde le dictionnaire mis à jour
    with open('dictionnaire_following-v2.json', 'w') as fp:
        json.dump(dictionnaire_following, fp)
    
    if not(continuer) : break

    # on doit attendre 15 minutes avant de pouvoir faire de nouvelles requêtes
    time.sleep(900)